In [1]:
import cv2 as cv
import numpy as np 

import matplotlib.pyplot as plt

import os
from Map import Map

In [2]:
PATH_TO_FOLDER = "./SrcData"
SIZE_OF_SHEET = [453, 465]
CELL_SIZE = 5

In [3]:
SQUARES_INDEX = {1: ['A', 'a', '1', '74'], 2: ['B', 'b', '2', '75'], 3:['C', 'c', '3', '86'], 4: ['D', 'd', '4', '87']}

In [4]:
def GetSheet(path: str):

    _map = Map(path)
    
    if _map.GetCellSize() != CELL_SIZE:

        _map.SetCellSize(CELL_SIZE)

    _map.SetMapSize(SIZE_OF_SHEET)

    return _map.GetData()

In [5]:
def unique(srcList):
    
    result = []
    for element in srcList:
        if (element in result) is False:
            result.append(element)
            
    return result

def GetSquareIndex(value, squaresIndex):

    for key, squareIndex in zip(squaresIndex.keys(), squaresIndex.values()):
        if value in squareIndex:
            return key

    return 1

    # concat squares
def GetDiamension(squares, axis):

    if axis == 0:
        table1 = 1
        table2 = 2
        table3 = 3
        table4 = 4
    else:   
        table1 = 1
        table2 = 3
        table3 = 2
        table4 = 4
    #get diamensions
    resultRange = -1
    resultRangeLeft = -1
    resultRangeRight = -1


    if table1 in squares and table3 in squares:
        resultRange = squares[table1].shape[axis] + squares[table3].shape[axis]
    elif table1 in squares:
        resultRangeLeft = squares[table1].shape[axis]*2
    elif table3 in squares:
        resultRangeLeft = squares[table3].shape[axis]*2

    if table2 in squares and table4 in squares:
        if resultRange < squares[table2].shape[axis] + squares[table4].shape[axis]:
            resultRange = squares[table2].shape[axis] + squares[table4].shape[axis]
    elif table2 in squares:
        resultRangeRight = squares[table2].shape[axis]*2
    elif table4 in squares:
        resultRangeRight = squares[table4].shape[axis]*2
        
    
    if resultRange == -1:
        if resultRangeLeft != -1:
            resultRange = resultRangeLeft
        if resultRangeRight != -1:
            if resultRange > resultRangeRight or resultRange == -1:
                resultRange = resultRangeRight

    return resultRange

In [6]:
def InsertSquaresIntoBiggerSquare(squares, square):
    rows, cols = square.shape
    centreX, centreY = int(rows/2), int(cols/2)
    if 1 in squares:
        if centreX < squares[1].shape[0]:
            beginX = 0
            endX = squares[1].shape[0]
        else:
            beginX = centreX - squares[1].shape[0]
            endX = centreX

        if centreY < squares[1].shape[1]:
            beginY = 0
            endY = squares[1].shape[1]
        else:
            beginY = centreY - squares[1].shape[1]
            endY = centreY
        square[beginX:endX,beginY:endY] = squares[1]
    if 2 in squares:
        if centreX < squares[2].shape[0]:
            beginX = 0
            endX = squares[2].shape[0]
        else:
            beginX = centreX - squares[2].shape[0]
            endX = centreX

        if square.shape[1] - centreY < squares[2].shape[1]:
            beginY = square.shape[1] - squares[2].shape[1]
            endY = square.shape[1]
        else:
            beginY = centreY
            endY = centreY + squares[2].shape[1]

        square[beginX:endX,beginY:endY] = squares[2]
    if 3 in squares:

        if square.shape[0] - centreX < squares[3].shape[0]:
            beginX = square.shape[0] - squares[3].shape[0]
            endX = square.shape[0]
        else:
            beginX = centreX
            endX = centreX + squares[3].shape[0]

        if centreY < squares[3].shape[1]:
            beginY = 0
            endY = squares[3].shape[1]
        else:
            beginY = centreY - squares[3].shape[1]
            endY = centreY
        square[beginX:endX,beginY:endY] = squares[3]
    if 4 in squares:
        if square.shape[0] - centreX < squares[4].shape[0]:
            beginX = square.shape[0] - squares[4].shape[0]
            endX = square.shape[0]
        else:
            beginX = centreX
            endX = centreX + squares[4].shape[0]

        if square.shape[1] - centreY < squares[4].shape[1]:
            beginY = square.shape[1] - squares[4].shape[1]
            endY = square.shape[1]
        else:
            beginY = centreY
            endY = centreY + squares[4].shape[1]
        
        square[beginX:endX, beginY:endY] = squares[4]
    return square

In [7]:

def GetSheetMap(filePaths, index):
    #filter by square milion
    coordValues = [filePath["coords"][index]  for filePath in filePaths.values()]
    coordValues = unique(coordValues)
    coordValues.sort()

    squares = {}
    if index == 6:
        for i, filePath in zip(filePaths.keys(), filePaths.values()):
            square = GetSheet(filePath["path"])
            squares[GetSquareIndex(filePath["coords"][index], SQUARES_INDEX)] = square
    else:
        for coordValue in coordValues:
            filtredFilePaths= {i: filePath for i, filePath in zip(filePaths.keys(), filePaths.values()) if filePath["coords"][index] == coordValue}
            square = GetSheetMap(filtredFilePaths, index + 1) 
            squares[GetSquareIndex(coordValue, SQUARES_INDEX)] = square


    xRange = GetDiamension(squares, 0)
    yRange = GetDiamension(squares, 1)

    biggerSquare = np.zeros((xRange, yRange))
    biggerSquare[:,:] = np.nan
    biggerSquare = InsertSquaresIntoBiggerSquare(squares, biggerSquare)
    return biggerSquare


In [8]:
filePaths = os.listdir(PATH_TO_FOLDER)

filePaths.sort()
filePaths = {i: {"path": os.path.join(PATH_TO_FOLDER, path), "coords": path.split(".")[0].split("-")}  for i, path in enumerate(filePaths)}

In [9]:
data = GetSheetMap(filePaths, 2)

### Save map

In [11]:
data = np.nan_to_num(data, nan=-1)
data = data[~np.all(data == -1, axis=1)]
# Usunięcie kolumn wypełnionych tylko zerami
data = data[:, ~np.all(data == -1, axis=0)]
data[data== -1] = np.nan

np.save("Data/ConcatedMap.npy", data)

In [ ]:
data = np.nan_to_num(data, nan=0)
plt.imsave("Results/ConcatedMap.png", data)